In [1]:
import pandas as pd
import json

In [2]:
with open("data/sample_data1.json") as f:
  data = json.load(f)

#print(data['tickets'])
#tickets_json = json.dumps(data.tickets)
#print(data['tickets'])
tickets = {}
tickets = data['tickets']

with open('data/ticket_data.json', 'w') as json_file:
  json.dump(tickets, json_file)


Manually remove first and last brackets from the json file

In [4]:
# read json into a dataframe
df_idf = pd.read_json("data/ticket_data.json",lines=True)

# print schema
print("Schema:\n\n",df_idf.dtypes)
print("Number of questions,columns=",df_idf.shape)

Schema:

 ticket_id             int64
title                object
content              object
timestamp    datetime64[ns]
tags                 object
dtype: object
Number of questions,columns= (31794, 5)


In [5]:
import re
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    return text

df_idf['text'] = df_idf['title'] + df_idf['content']
df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))

#show the first 'text'
df_idf['text'][0]

'firefox keeps going back to previous page randomly but i have not changed anything i have the same os and all else is same but recently firefox keeps going back to the previous page randomly and i have to click that forward arrow to get it back a real pain as it reloads and i have to start over from what i was reading copy pasting etc perhaps it s a bug with a recent firefox update '

# Creating the IDF

##  CountVectorizer to create a vocabulary and generate word counts

In [6]:
#import nltk
#nltk.download('stopwords')

import nltk
from nltk.corpus import stopwords

print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
import re

#get the text column 
docs=df_idf['text'].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.85,stop_words=stopwords.words('english'), min_df=1)
word_count_vector=cv.fit_transform(docs)

In [8]:
word_count_vector.shape

(31794, 45336)

In [9]:
list(cv.vocabulary_.keys())[:10]

['firefox',
 'keeps',
 'going',
 'back',
 'previous',
 'page',
 'randomly',
 'changed',
 'anything',
 'os']

## TfidfTransformer to Compute Inverse Document Frequency (IDF)

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [11]:
tfidf_transformer.idf_

array([ 9.57530485,  9.57530485, 10.26845204, ..., 10.67391714,
       10.67391714, 10.67391714])

# Computing TF-IDF and Extracting Keywords

Get first 500 tickets from the json

In [12]:
with open("data/sample_data1.json") as f:
  data = json.load(f)

tickets = {}
tickets = data['tickets']

sample_tickets = []
sample_tickets =  tickets[0:500]

#print(sample_tickets)

with open('data/sample_ticket_data.json', 'w') as json_file:
  json.dump(sample_tickets, json_file)



Manually remove first and last brackets from the json file

In [13]:
# read test docs into a dataframe and concatenate title and body
df_test=pd.read_json("data/sample_ticket_data.json",lines=True)
df_test['text'] = df_test['title'] + df_test['content']
df_test['text'] =df_test['text'].apply(lambda x:pre_process(x))

# get test docs into a list
docs_test=df_test['text'].tolist()
docs_title=df_test['title'].tolist()
docs_body=df_test['content'].tolist()

In [14]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [15]:
# you only needs to do this once
feature_names=cv.get_feature_names()

# get the document that we want to extract keywords from
doc=docs_test[0]

#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)

# now print the results
print("\n=====Title=====")
print(docs_title[0])
print("\n=====Body=====")
print(docs_body[0])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Title=====
firefox keeps going back to previous page randomly , but i have not changed anything

=====Body=====
<p>i have the same OS and all else is same, but recently firefox keeps going back to the previous page randomly , and i have to click that forward arrow to get it back. a real pain as it reloads and i have to start over from what i was reading/copy-pasting etc.
Perhaps it's a bug with a recent firefox update?
</p>

===Keywords===
randomly 0.367
back 0.287
keeps 0.269
previous 0.268
going 0.259
reloads 0.237
pasting 0.233
pain 0.209
perhaps 0.195
reading 0.185


In [16]:
# put the common code into several methods
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs_test[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords):
    # now print the results
    print("\n=====Title=====")
    print(docs_title[idx])
    print("\n=====Body=====")
    print(docs_body[idx])
    print("\n===Keywords===")
    for k in keywords:
        print(k,keywords[k])

In [17]:
idx=135
keywords=get_keywords(idx)
print_results(idx,keywords)


=====Title=====
Every time I look at my history the first thing I see is a some Russian website called yandex.ru. I didn't even visit it.

=====Body=====
<p>The only thing I did, was install the yandex toolbar, while installing other programs. A lot of bookmarks showed up, but I deleted them. There is no program called yandex or something in my control panel (install unistall programs). I really don't want to reinstall firefox.
</p>

===Keywords===
yandex 0.641
called 0.258
programs 0.249
thing 0.216
russian 0.197
unistall 0.195
install 0.194
ru 0.179
showed 0.157
panel 0.143
